In [25]:
import pandas
from ipywidgets import *
import altair as alt

Load the two CSV files from DUO open data.

The data was downlaoded from the DUO [open data site](https://duo.nl/open_onderwijsdata/databestanden/). The meaning of the codes used in the files are explained in the 'toelichting' files that are available (e.g. for [schooladviezen](https://duo.nl/open_onderwijsdata/images/toelichting-04-schooladviezen.pdf) file).


In [34]:
with open('data/duo/05.-gemiddelde-afstand-tussen-woonadres-leerling-en-schoolvestiging-2017-2018.csv', 'r', encoding='Windows-1252') as csvfile:
    afstand_data = pandas.read_csv(csvfile, sep=';',  quotechar="\"", decimal=",")
    afstand_data.set_index(['BRIN_NUMMER', 'VESTIGINGSNUMMER'])
    # Filter primary schools only (no 'bijzonder onderwijs')
    afstand_data = afstand_data[afstand_data['SOORT_PO'] == 'Bo']

In [35]:
with open('data/duo/04.-schooladviezen-2016-2017.csv', 'r', encoding='Windows-1252') as csvfile:
    data = pandas.read_csv(csvfile, sep=';',  quotechar="\"")
    data.set_index(['BRIN_NUMMER', 'VESTIGINGSNUMMER'])
    # Filter primary schools only (no 'bijzonder onderwijs')
    data = data[data['SOORT_PO'] == 'Bo']
    # Calculate the fraction of VWO advies out of the total
    data['vwo_frac'] = data['VWO']/data['TOTAAL_ADVIES']
    

In [36]:
# Join the two datasets
advies_afstand = data.join(afstand_data, lsuffix='_advies', rsuffix='_afstand')

In [40]:
# Widget voor plaatsnaam
plaatsnamen = afstand_data.PLAATSNAAM.unique()
plaatsnamen.sort()
select_plaatsnaam = widgets.Select(
    options=list(plaatsnamen),
    value="AMSTERDAM",
    description='Plaats:',
    disabled=False
)
#display(select_plaatsnaam)


In [41]:
def plaatsnaam_plot(plaatsnaam):
    in_plaats = advies_afstand[advies_afstand.PLAATSNAAM_afstand == plaatsnaam]
    c = alt.Chart(in_plaats).mark_circle().encode(
        x='AFSTAND:Q',
        y='vwo_frac',
        color='DENOMINATIE_VESTIGING_advies',
        size='VWO'
    )
    display(c)

In [42]:
interact(plaatsnaam_plot, plaatsnaam=select_plaatsnaam)

interactive(children=(Select(description='Plaats:', index=69, options=("'S GRAVENMOER", "'S-GRAVENDEEL", "'S-G…

<function __main__.plaatsnaam_plot(plaatsnaam)>

This can be extended using reveal.js to create presentations easily. This will only work in Jupyter Noteboook using the [RISE](https://github.com/damianavila/RISE) plugin.